# Main ...

 
``config, load, preprocess, train, eval  for  Tick tick bloom``

**Yo DON'T rerun this unless you want to overwrite past models, always fork and do your stuff and 
DON'T forget to change the name**

**``Mission: NNs on expanding avg of severity by region metadata``**

# Load imports and dependencies

In [2]:
import warnings
import sys
import os
import time
import joblib
import random
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, activations, losses, metrics, models, optimizers, callbacks
from category_encoders.target_encoder import TargetEncoder

warnings.filterwarnings('ignore')

In [3]:
# local utilities imports
from tick_tick_bloom_utils import my_keras_rmse, comp_metric, den2sev_map

In [4]:
# wandb stuff for tracking
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_login = user_secrets.get_secret("wandb_bloom_tracker")

import wandb
wandb.login(key=wandb_login)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Config

In [ ]:
# dot dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


# Config
config = {}
config = dotdict(config)
config['RANDOM_SEED'] = 18952


config['unique_id'] = int(time.time())
print(f'unique_id: {config.unique_id}')
config['name'] = f'd128_rmse_mtdata-{config.unique_id}'

config['PROJECT_NAME'] = 'tick-tick-bloom'
# config['DATA_DIR'] = '../data/'
# config['MODEL_DIR'] = '../models/'
config['SAVE_MODEL'] = True


# # Img config
# config['IMG_SIZE'] = (136, 136)
# config['CHANNELS'] = 3

# training configuration
config['train'] =  dotdict({
                        'epochs': 50,
                        'batch_size': 128,
                        'validation_split': 0.2,
                        'shuffle': True,
                        'verbose': 1
                        })

config['desc'] = """
xp3-to see how nns perform only on metadata(use data upto the point of availibility.) with rmse loss monitor my_keras_rmse
Now modelling nn in traidtional form with sparse categorical loss and 5 out neruons
""" 

In [88]:
# seed everything
random.seed(config.RANDOM_SEED)
np.random.seed(config.RANDOM_SEED)
tf.random.set_seed(config.RANDOM_SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(config.RANDOM_SEED)

# tf.keras.utils.set_random_seed(config.RANDOM_SEED)  # supposedly sets seed for python, numpy, tf

# Load data

In [7]:
INPUT_METADATA_DIR = '/kaggle/input/ticktickbloomdataset'

metadata = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'metadata.csv'))
sub_format = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'submission_format.csv'))
train_labels = pd.read_csv(os.path.join(INPUT_METADATA_DIR, 'train_labels.csv'))

In [8]:
# IMG_DIR = '/kaggle/input/pull-landsat-data-v1-500m/landsat8_500m_v1'   # landsat 8 data with raw 
# img_files = os.listdir(IMG_DIR)
# img_file_names = [f.split('.')[0] for f in img_files]

# # get only data for those 1k imgs
# metadata_subset = metadata[metadata['uid'].isin(img_file_names)]
# data = metadata_subset[metadata_subset.split == 'train']
# data = data.merge(train_labels, on='uid')

# test_data = metadata[metadata.split == 'train']

In [9]:
# def get_imgs(uids) :
#     imgs = []
#     for uid in uids:
#         img_arr = np.load(IMG_DIR + f'/{uid}.npy')
#         img_arr = np.transpose(img_arr, (2, 1, 0))
#         # resize img
#         img_arr = cv2.resize(img_arr, config.IMG_SIZE)
#         img_arr = img_arr / 255   # normalizeee bro... other wise it's blowing up the networks...
#         imgs.append(img_arr)
#     return np.array(imgs) 


# def get_np_data(split : float = 0.2, task='train'):
#     """Return np data for training and validation."""
#     if task == 'train':
#         print("Loading train and validation data...")
#         x_train_uids, x_val_uids, y_train, y_val = train_val_split(
#             data['uid'],
#             data.severity,
#             val_size=split,
#             random_state=config.RANDOM_SEED,
#             stratify=data.severity
#         )

#         x_train = get_imgs(x_train_uids)
#         x_val = get_imgs(x_val_uids)

#         return x_train, y_train, x_val, y_val


#     if task == 'test':
#         test_ids = test_data.uids
#         x_test
#         return x_test

In [10]:
# x_train, y_train, x_val, y_val = get_np_data()
# print(y_train.value_counts(normalize=True))
# print(y_val.value_counts(normalize=True))
# print('Done')

In [11]:
#  get data
metadata.date = pd.to_datetime(metadata.date)

region = pd.concat((train_labels, sub_format[['region', 'uid']]), axis=0)

data = pd.merge(metadata, region, on='uid', how='left')
print(data.shape)

(23570, 8)


In [12]:
# seasons
seasons = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1
}


#  most of the samples are collected in the months of June, July, August.

# add date time fts.
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data['week'] = data.date.dt.isocalendar().week
# data['day_of_year'] = data.date.dt.
data['season'] = data.month.map(seasons)

In [13]:
data.sort_values(by='date', inplace=True)

In [15]:
data['expanding_severity'] = data.severity.expanding().mean()
data['expanding_severity'] = data['expanding_severity'].apply(np.round)
data

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity
4387,evep,44.847993,-93.476318,2013-01-04,train,midwest,1.0,115.0,1,2013,1,1,1.0
13644,paev,44.822478,-93.367962,2013-01-04,train,midwest,1.0,1884.0,1,2013,1,1,1.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,midwest,1.0,1416.0,1,2013,1,1,1.0
6144,guny,44.878889,-93.490833,2013-01-04,train,midwest,1.0,558.0,1,2013,1,1,1.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,midwest,1.0,476.0,1,2013,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0
17559,thki,36.725400,-121.730000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0
17452,teuu,36.772300,-121.788000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0
14254,prfi,36.751800,-121.742000,2021-12-29,test,west,NaN,NaN,12,2021,52,1,2.0


In [16]:
train_data = data[data.split == 'train']
test_data = data[data.split == 'test']

In [17]:
# expanding avg of severity
mse(train_data.severity, train_data.expanding_severity, squared=False)

1.2085810811762927

In [18]:
grp_by_region = data.groupby('region').severity.expanding(1).mean()
grp_by_region = grp_by_region.map(np.round)

grp_by_region['west'].fillna(2, inplace=True)
grp_by_region['northeast'].fillna(2, inplace=True)
print(grp_by_region.isna().sum())   # 5 --> 0.89416

print(mse(train_data.severity.sort_index(), grp_by_region.droplevel(0).loc[train_data.index].sort_index(), squared=False))

data['expndng_sev_by_reg'] = np.nan

south = data.region == 'south'
midwest = data.region == 'midwest'
northeast = data.region == 'northeast'
west = data.region == 'west'

data.loc[south , 'expndng_sev_by_reg'] = grp_by_region['south']
data.loc[midwest , 'expndng_sev_by_reg'] = grp_by_region['midwest']
data.loc[northeast , 'expndng_sev_by_reg'] = grp_by_region['northeast']
data.loc[west , 'expndng_sev_by_reg'] = grp_by_region['west']

print(data.shape)
data.isna().sum()

data.sort_index()

0
0.894165010958815
(23570, 14)


,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity,expndng_sev_by_reg
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1.0,585.0,5,2018,20,2,2.0,2.0
1,aabn,36.559700,-121.510000,2016-08-31,test,west,NaN,NaN,8,2016,35,3,2.0,4.0
2,aacd,35.875083,-78.878434,2020-11-19,train,south,1.0,290.0,11,2020,47,4,2.0,2.0
3,aaee,35.487000,-79.062133,2016-08-24,train,south,1.0,1614.0,8,2016,34,3,2.0,2.0
4,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3.0,111825.0,7,2019,30,3,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23565,zzvv,36.708500,-121.749000,2014-12-02,test,west,NaN,NaN,12,2014,49,1,2.0,4.0
23566,zzwo,39.792190,-99.971050,2017-06-19,train,midwest,2.0,48510.0,6,2017,25,3,2.0,2.0
23567,zzwq,35.794000,-79.012551,2015-03-24,train,south,1.0,1271.0,3,2015,13,2,2.0,1.0
23568,zzyb,35.742000,-79.238600,2016-11-21,train,south,1.0,9682.0,11,2016,47,4,2.0,2.0


In [65]:
# # MY ASSUMPTION: less the missing values --> more inital samples in the group are not test --> less imputations/ffills needed --> much realiable score!
# grp_by_rs = train_data.groupby(['region', 'season']).severity.expanding(1).mean()
# grp_by_rs = grp_by_rs.map(np.round)
# print(grp_by_rs.isna().sum()) # 5 --> .86
# mse(train_data.severity.sort_index(), grp_by_rs.droplevel(0).droplevel(0).sort_index()[:train_data.shape[0]], squared=False)

0


0.8594349134502333

In [20]:
grp_by_rs = data.groupby(['region', 'season']).severity.expanding(1).mean()
grp_by_rs = grp_by_rs.map(np.round)
print(grp_by_rs.isna().sum()) # 5 --> .86

data['expanding_sev_rs'] =  grp_by_rs.droplevel(0).droplevel(0).sort_index()
# fillna with expanding sev by region
data['expanding_sev_rs'] = np.where(data.expanding_sev_rs.isna(), data.expndng_sev_by_reg, data.expanding_sev_rs)

print(mse(train_data.severity.sort_index(), data['expanding_sev_rs'].sort_index()[data.split == 'train'], squared=False))

# #  make submission for expanding severity by region and season

# expanding_sev_rs = data[data.split == 'test'][['uid', 'expanding_sev_rs']]          # picking up only uids and expanding_sev_rs from test samples
# expanding_sev_rs.expanding_sev_rs = expanding_sev_rs.expanding_sev_rs.astype(int)   # casting to int
# expanding_sev_rs.sort_values(by='uid', inplace=True)                                # sorting by uid -- safest option
# expanding_sev_rs.reset_index(drop=True, inplace=True)                               # matching indexes with submissoin

# sub_format.severity = expanding_sev_rs.expanding_sev_rs
# sub_format.severity.value_counts()  # expected 0.8594349134502333

# sub_format.to_csv('expanding_sev_rs_preds.csv', index=False)

5
0.8594349134502333


In [105]:
train_labels.severity.value_counts(normalize=True)
# since test and train dists are almost similar my ideal model should follow this dist!

1    0.439449
4    0.207913
2    0.189859
3    0.159379
5    0.003400
Name: severity, dtype: float64

In [22]:
data.head()

,uid,latitude,longitude,date,split,region,severity,density,month,year,week,season,expanding_severity,expndng_sev_by_reg,expanding_sev_rs
4387,evep,44.847993,-93.476318,2013-01-04,train,midwest,1.0,115.0,1,2013,1,1,1.0,1.0,1.0
13644,paev,44.822478,-93.367962,2013-01-04,train,midwest,1.0,1884.0,1,2013,1,1,1.0,1.0,1.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,midwest,1.0,1416.0,1,2013,1,1,1.0,1.0,1.0
6144,guny,44.878889,-93.490833,2013-01-04,train,midwest,1.0,558.0,1,2013,1,1,1.0,1.0,1.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,midwest,1.0,476.0,1,2013,1,1,1.0,1.0,1.0


In [23]:
data.isna().sum()

uid                      0
latitude                 0
longitude                0
date                     0
split                    0
region                   0
severity              6510
density               6510
month                    0
year                     0
week                     0
season                   0
expanding_severity       0
expndng_sev_by_reg       0
expanding_sev_rs         0
dtype: int64

In [24]:
all_train = data[data.split == 'train']
all_train.shape

(17060, 15)

In [25]:
test_data = metadata[metadata.split == 'test']
test_data = test_data.merge(sub_format, on='uid')

test_data['month'] = test_data.date.dt.month
test_data['year'] = test_data.date.dt.year
test_data['season'] = test_data.month.map(seasons)

In [26]:
train, val = train_test_split(all_train, test_size=0.20, random_state=config.RANDOM_SEED, stratify=all_train.severity)
train.shape, val.shape

((13648, 15), (3412, 15))

In [29]:
all_train.columns

Index(['uid', 'latitude', 'longitude', 'date', 'split', 'region', 'severity',
       'density', 'month', 'year', 'week', 'season', 'expanding_severity',
       'expndng_sev_by_reg', 'expanding_sev_rs'],
      dtype='object')

In [44]:
req_cols = ['region', 'month', 'year', 'season', 'expndng_sev_by_reg']

In [45]:
X_train_, y_train_ = train[req_cols], train['severity'] 
X_val_, y_val_ = val[req_cols], val['severity']

X_train_.shape, y_train_.shape, X_val_.shape, y_val_.shape

((13648, 5), (13648,), (3412, 5), (3412,))

In [55]:
test_data['expanding_severity'] = data[data.split == 'test']['expanding_severity'].sort_index().values
test_data['expndng_sev_by_reg'] = data[data.split == 'test']['expndng_sev_by_reg'].sort_index().values
test_data['expanding_sev_rs'] = data[data.split == 'test']['expanding_sev_rs'].sort_index().values

In [56]:
X_test_ = test_data[req_cols]
X_test_.shape

(6510, 5)

# Preprocess

In [57]:
# # change labels to 0-3(model works this way) instead of 1-4 given range(given severity)
# -1 for to make labels look like sparse encoded labels

y_train = y_train_  -1
y_val = y_val_ - 1

y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)  # guessing alwyas 0 gives 43% acc

(0.0    0.439478
 3.0    0.207943
 1.0    0.189845
 2.0    0.159364
 4.0    0.003370
 Name: severity, dtype: float64,
 0.0    0.439332
 3.0    0.207796
 1.0    0.189918
 2.0    0.159437
 4.0    0.003517
 Name: severity, dtype: float64)

In [58]:
# #  target encode the cat fts.

# te = TargetEncoder(cols=['region', 'month', 'year', 'season'])
# te.fit(X_train_, y_train)
# X_train =  te.transform(X_train_)
# X_val = te.transform(X_val_)

# X_test = te.transform(X_test_)
# X_test

In [59]:
X_train_.dtypes

region                 object
month                   int64
year                    int64
season                  int64
expndng_sev_by_reg    float64
dtype: object

In [60]:
#  Encode year

from category_encoders.ordinal import OrdinalEncoder

oe = OrdinalEncoder(cols=['year', 'region'])
X_train = oe.fit_transform(X_train_)
X_test = oe.transform(X_test_)
X_val = oe.transform(X_val_)

# Model

In [63]:
def comp_loss(y_true, y_pred):
    return tf.sqrt(losses.mean_squared_error(y_true, y_pred))

In [89]:
def get_model():
    print('Loading model...')
#     input_shape = (*config.IMG_SIZE, config.CHANNELS)

    input_imgs = layers.Input(shape=(X_train.shape[1],))
#     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Conv2D(32, (3, 3), activation='relu')(input_imgs)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(input_imgs)
    output = layers.Dense(1)(x)     # regressing the severity level !

    model = models.Model(inputs=input_imgs, outputs=output, name=config.name)

    model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
                    loss=my_keras_rmse,
                    metrics=[
                        my_keras_rmse,
                        metrics.SparseCategoricalAccuracy(name='acc')
                    ])

    return model

In [90]:
model = get_model()
model.summary()

Loading model...
Model: "d128_rmse_mtdata-1673416598"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               768       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.fit(X_train, y_train)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.7/site-pack

# Train and eval

In [80]:
def train(model=get_model(), config=config, x_train=X_train, y_train=y_train):
    """fits given model to x_train and y_train"""
    
    train_config = config['train']
    my_callbacks = []
    
    earlystopping = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    my_callbacks.append(earlystopping)
    
    try:
        wandb_callback = wandb.keras.WandbCallback(
            monitor='val_loss',
            log_weights=True,
            log_gradients=True,
            save_model=False,
            training_data=(x_train, y_train),
            log_batch_frequency=None,
        )

        my_callbacks.append(wandb_callback)
    except:
        print('wandb not tracking')
        
    print(f'Training model... {config.name}')
    history = model.fit(
                x_train, y_train,
                epochs=train_config.epochs,
                batch_size=train_config.batch_size, 
                callbacks=my_callbacks, 
                validation_split=0.2, 
                shuffle=True, 
                verbose=1   
            )

    return model, history

Loading model...


In [81]:
def eval_model(model, x_val=X_val, y_val=y_val):
    print('Evaluating model....')
    model.evaluate(x_val, y_val, return_dict=True)


In [82]:
with wandb.init(project=config.PROJECT_NAME, config=config, name=config.name):
    
    # load nwe model or old saved model
    model = get_model()

    # train
    model, history = train(model, config, y_train=y_train)
    # eval
    eval_model(model)
    
    # classification report
    y_pred = model.predict(X_val)
    y_pred_hard = np.round(y_pred)
    error = mse(y_val, y_pred_hard, squared=False)
    print("Comp Metric: ", error)
    cr = classification_report(y_val+1, y_pred_hard+1)     # +1 to account for 0-4 as it should be 1-5 originallly
    print(cr)
    

Loading model...
Training model... d128_rmse_mtdata-1673416438
Epoch 1/50


2023-01-11 05:54:49.906007: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-01-11 05:54:49.906277: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-11 05:54:49.907467: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.009ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



86/86 [==============================] - 2s 7ms/step - loss: 0.9946 - my_keras_rmse: 1.1445 - acc: 0.4392 - val_loss: 0.9555 - val_my_keras_rmse: 1.1513 - val_acc: 0.4407
Epoch 2/50
86/86 [==============================] - 0s 4ms/step - loss: 0.9199 - my_keras_rmse: 1.1445 - acc: 0.4392 - val_loss: 0.9141 - val_my_keras_rmse: 1.1513 - val_acc: 0.4407
Epoch 3/50
86/86 [==============================] - 0s 4ms/step - loss: 0.8761 - my_keras_rmse: 1.1445 - acc: 0.4392 - val_loss: 0.8680 - val_my_keras_rmse: 1.1513 - val_acc: 0.4407
Epoch 4/50
86/86 [==============================] - 0s 4ms/step - loss: 0.8291 - my_keras_rmse: 1.1445 - acc: 0.4392 - val_loss: 0.8190 - val_my_keras_rmse: 1.1513 - val_acc: 0.4407
Epoch 5/50
86/86 [==============================] - 0s 4ms/step - loss: 0.7845 - my_keras_rmse: 1.1445 - acc: 0.4392 - val_loss: 0.7760 - val_my_keras_rmse: 1.1513 - val_acc: 0.4407
Epoch 6/50
86/86 [==============================] - 0s 4ms/step - loss: 0.7435 - my_keras_rmse: 1.144

acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▆▅▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
my_keras_rmse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▅▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_my_keras_rmse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPS,0.0
acc,0.43918
best_epoch,49
best_val_loss,0.62159


In [ ]:
# history.history

In [ ]:
# model = models.load_model('/kaggle/working/d128_rmse_lndsat8_raw_v1-1673283452.h5', custom_objects={'comp_loss': comp_loss})
# preds = model.predict(X_val)
# int_preds = np.round(preds)
# mse(y_val, int_preds, squared=False)

# Save something..

In [83]:
# save model
if config.SAVE_MODEL:
    model.save(config.name + '.h5')
    print("Model saved as ",config.name + '.h5')

Model saved as  d128_rmse_mtdata-1673416438.h5


# Make submission

In [84]:
model.summary()

Model: "d128_rmse_mtdata-1673416438"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               768       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


In [85]:
test_preds = np.round(model.predict(X_test)).ravel()
test_preds = test_preds + 1
sub_format.severity = test_preds
sub_format.severity = sub_format.severity.astype(int) 
sub_format.severity.value_counts()

2    3273
4    2386
1     851
Name: severity, dtype: int64

In [86]:
sub_format.to_csv(f'{config.name}_preds.csv', index=False) # expecting @ 0.9777

# So...

- Just realized that should only use data upto point of availibility.
- Expanding avg severity by region @ 0.89 best


# ToDos:

- IS this way of modelling with 1 output neuron better compared to 5 as clf
- Does adding data provide any value??
- **Try to beat expanding avg_severity_by_region with the help of imgs, Other wise no use for img data**
- Is there any other way to extract more info than expanding_avg_severity_by_region only from metadata???